In [38]:
import pandas as pd
import numpy as np
import os
import json
import time
import datetime
from dateutil.relativedelta import relativedelta

In [106]:
# for f in os.listdir('raw_data'):
#     filename = 'raw_data/' + f
#     with open(filename, 'r') as contents:
#         wine_reviews = json.loads(contents)

with open('raw_data/4057966.json') as filepath:
    wine_reviews = json.loads(filepath.read())

wine_review_df = pd.json_normalize(wine_reviews)
scrape_date_unix = os.path.getmtime('raw_data/4057966.json')
scrape_date = datetime.datetime.fromtimestamp(scrape_date_unix)

In [115]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


def compute_date(scrape_date, review_date, review_time_ago):
    review_month = review_date[5:8]
    review_day = review_date[-20:-18].strip()

    if 'over' in review_time_ago:
        crop_offset_string = review_time_ago.split('over')[1].strip()
        offset_period = int(crop_offset_string[:2].strip())

        min_date = scrape_date - relativedelta(months=12*(offset_period+1))
        max_date = scrape_date - relativedelta(months=12*offset_period)

        candidate_years = [min_date.year, max_date.year]
        candidate_dates = [datetime.datetime.strptime(review_day + ' ' + review_month + ' ' + str(y), '%d %b %Y') for y in candidate_years]

        final_review_date = [d for d in candidate_dates if d > min_date and d < max_date][0]

    elif 'almost' in review_time_ago:
        crop_offset_string = review_time_ago.split('almost')[1].strip()
        offset_period = int(crop_offset_string[:2].strip())

        min_date = scrape_date - relativedelta(months=12*offset_period)
        max_date = scrape_date

        candidate_years = [min_date.year, max_date.year]
        candidate_dates = [datetime.datetime.strptime(review_day + ' ' + review_month + ' ' + str(y), '%d %b %Y') for y in candidate_years]

        final_review_date = [d for d in candidate_dates if d > min_date and d < max_date][0]      

    else:
        if 'about' in review_time_ago:
            crop_offset_string = review_time_ago.split('about')[1].strip()
            offset_period = int(crop_offset_string[:2].strip())
        else:
            offset_period = int(review_time_ago[:2].strip())

        if 'month' in review_time_ago:
            offset_scrape_date = scrape_date - relativedelta(months=offset_period)
        elif 'year' in review_time_ago:
            offset_scrape_date = scrape_date - relativedelta(months=12*offset_period)
        else:
            offset_scrape_date = scrape_date
        
        candidate_years = [offset_scrape_date.year - 1, offset_scrape_date.year, offset_scrape_date.year + 1]
        try:
            candidate_dates = [datetime.datetime.strptime(review_day + ' ' + review_month + ' ' + str(y), '%d %b %Y') for y in candidate_years]
        # in some fringe cases, we may be dealing with February 29th, which only exists on leap years
        except ValueError:
            candidate_dates = [datetime.datetime.strptime(str(int(review_day) - 1) + ' ' + review_month + ' ' + str(y), '%d %b %Y') for y in candidate_years]
        
        final_review_date = nearest(candidate_dates, offset_scrape_date)
    
    return final_review_date


# function to create a compound ID that uniquely identifies a wine by its vintage, review year 
def create_wine_year_id(wine_id, vintage, review_year):
    compound_id = str(wine_id) + '-' + str(vintage) + '-' + str(review_year)
    return compound_id


def clean_wine_reviews(review_df):
    review_df['final_review_date'] = review_df.apply(lambda x: compute_date(scrape_date, x['review_date'], x['review_time_ago']), axis=1)
    review_df['review_year'] = review_df['final_review_date'].apply(lambda x: x.year)

    # drop any reviews that don't have a vintage specified. N.V. is acceptable, but blank vintage is not. 
    review_df['vintage'].replace({'': np.nan}, inplace=True)
    review_df.dropna(subset=['vintage'], axis=0, inplace=True)

    review_df['wine_year_id'] = review_df.apply(lambda x: create_wine_year_id(x['wine_id'], x['vintage'], x['review_year']), axis=1)

    just_reviews = review_df[['wine_year_id',  'review_year', 'wine_id', 'rating', 'final_review_date']]
    return just_reviews


In [120]:

from itertools import combinations

wine_reviews = clean_wine_reviews(wine_review_df)
wine_reviews = wine_reviews.set_index(['wine_year_id'])

def compute_head_to_head_result(wine_0, wine_1, rating_0, rating_1):
    if rating_0 > rating_1:
        return wine_0
    elif rating_0 < rating_1:
        return wine_1
    else:
        return np.nan


def elo_transform(review_df):
    review_years = list(set(review_df['final_review_date']))
    head_to_heads = []
    for r in review_years:
        review_df_slice = review_df.loc[review_df['final_review_date'] == r]
        # in case there are duplicate reviews of a wine within a given year, only keep the first one
        review_df_slice = review_df_slice.sort_values(by='final_review_date')
        review_df_slice = review_df_slice.reset_index().drop_duplicates(subset='wine_year_id', keep='first').set_index('wine_year_id')
        combo = list(combinations(review_df_slice.index, 2))
        for c in combo:
            rating_0 = review_df_slice.at[c[0], 'rating']
            rating_1 = review_df_slice.at[c[1], 'rating']
            result = compute_head_to_head_result(c[0], c[1], rating_0, rating_1)
            date_0 = review_df_slice.at[c[0], 'final_review_date']
            date_1 = review_df_slice.at[c[1], 'final_review_date']
            head_to_head_date = max([date_0, date_1])

            output = [c[0], c[1], result, head_to_head_date]

            head_to_heads.append(output)
    
    return head_to_heads

results = pd.DataFrame(elo_transform(wine_reviews), columns=['wine_id_0', 'wine_id_1', 'result', 'date'])
print(results.head(20))

              wine_id_0            wine_id_1               result       date
0   128488603-2014-2016    6005359-2013-2016  128488603-2014-2016 2016-05-24
1   128488603-2014-2016  152865445-2014-2016  128488603-2014-2016 2016-05-24
2     6005359-2013-2016  152865445-2014-2016    6005359-2013-2016 2016-05-24
3     1461803-2011-2019    2902509-2005-2019    2902509-2005-2019 2019-06-26
4     1461803-2011-2019  152489457-2017-2019  152489457-2017-2019 2019-06-26
5     2902509-2005-2019  152489457-2017-2019  152489457-2017-2019 2019-06-26
6     2273741-2012-2014    4342756-2012-2014                  NaN 2014-08-30
7     7758438-2013-2015    3482061-2012-2015                  NaN 2015-12-06
8     7758438-2013-2015    1239962-2011-2015    1239962-2011-2015 2015-12-06
9     7758438-2013-2015    4172365-2012-2015    4172365-2012-2015 2015-12-06
10    7758438-2013-2015   23097634-2014-2015   23097634-2014-2015 2015-12-06
11    3482061-2012-2015    1239962-2011-2015    1239962-2011-2015 2015-12-06